In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import math

from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.callbacks import CSVLogger

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.cross_validation import  train_test_split
from sklearn.cross_validation import cross_val_score

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import scorer
from sklearn.metrics import accuracy_score

Using TensorFlow backend.
C:\Users\212469077\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
stock_name = 'ADBE'
sharpes = pd.DataFrame(columns=['STOCK','ANN','SVM','KNN','LR','acc_SVM_train','acc_SVM_test','acc_KNN_train','acc_KNN_test','acc_LR_train','acc_LR_test'])

In [3]:
df = pd.read_csv('all_stocks_5yr.csv')
df = df.drop(['volume'], axis=1)
stock_names = df.Name.unique()

In [4]:
def extract_stock(stock_name):
    df_stock = df.loc[df['Name'] == stock_name]
    df_stock = df_stock.drop(['Name'], axis=1)
    df_stock = df_stock.dropna()
    df_stock['date'] = pd.to_datetime(df_stock['date'])
    df_stock['close'] = pd.to_numeric(df_stock['close'], downcast='float')
    df_stock.set_index('date',inplace=True)
    df_stock.sort_index(inplace=True)
    
    return df_stock

In [ ]:
############################################################################################
################################### 0. ALL SCRIPTS START ###################################
############################################################################################

In [7]:
### ANN

#get stock
df_stock = extract_stock(stock_name)
    
#prepare variables
df_stock['H-L'] = df_stock['high'] - df_stock['low']
df_stock['O-C'] = df_stock['close'] - df_stock['open']
df_stock['3day MA'] = df_stock['close'].shift(1).rolling(window = 3).mean()
df_stock['10day MA'] = df_stock['close'].shift(1).rolling(window = 10).mean()
df_stock['30day MA'] = df_stock['close'].shift(1).rolling(window = 30).mean()
df_stock['Std_dev']= df_stock['close'].rolling(5).std()

#prepare predicted
df_stock['Price_Rise'] = np.where(df_stock['close'].shift(-1) > df_stock['close'], 1, 0)
    
#remove na after shift
df_stock = df_stock.dropna()
    
#define X
X = df_stock.iloc[:, 4:-1]
    
#define y
y = df_stock.iloc[:, -1]
    
#define split
split_percentage = 0.8
split = int(len(df_stock)*split_percentage)
    
#prepare train and test sets
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]
    
#define scaler
sc = StandardScaler()
    
#scale-replace X_train and X_test
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
    
#define model
classifier = Sequential()
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', input_dim = X.shape[1]))
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
    
#teach model
csv_logger = CSVLogger('log.csv', append=False, separator=',')
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, verbose = 1, callbacks=[csv_logger])

y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
    
df_stock['y_pred'] = np.NaN
df_stock.iloc[(len(df_stock) - len(y_pred)):,-1:] = y_pred
df_stock = df_stock.dropna()

#create metric

#calculate returns
df_stock['Tomorrows Returns'] = 0.
df_stock['Tomorrows Returns'] = np.log(df_stock['close']/df_stock['close'].shift(1))
df_stock['Tomorrows Returns'] = df_stock['Tomorrows Returns'].shift(-1)

df_stock['Strategy Returns'] = 0.
df_stock['Strategy Returns'] = np.where(df_stock['y_pred'] == True, df_stock['Tomorrows Returns'], - df_stock['Tomorrows Returns'])

df_stock['Cumulative Market Returns'] = np.cumsum(df_stock['Tomorrows Returns'])
df_stock['Cumulative Strategy Returns'] = np.cumsum(df_stock['Strategy Returns'])

Cumulative_Strategy_Returns = df_stock['Cumulative Strategy Returns']*100
Cumulative_Market_Returns = df_stock['Cumulative Market Returns']

Cumulative_ANN_Returns = Cumulative_Strategy_Returns

#calculate sharpe ratio
Std = Cumulative_Strategy_Returns.std()
Sharpe = (Cumulative_Strategy_Returns-Cumulative_Market_Returns)/Std
Sharpe_ANN = Sharpe.mean()

Epoch 1/100
983/983 [==============================] - 0s 389us/step - loss: 0.2503 - acc: 0.5046
Epoch 2/100
983/983 [==============================] - 0s 137us/step - loss: 0.2496 - acc: 0.5351
Epoch 3/100
983/983 [==============================] - 0s 132us/step - loss: 0.2492 - acc: 0.5351
Epoch 4/100
983/983 [==============================] - 0s 125us/step - loss: 0.2490 - acc: 0.5351
Epoch 5/100
983/983 [==============================] - 0s 132us/step - loss: 0.2487 - acc: 0.5351
Epoch 6/100
983/983 [==============================] - 0s 169us/step - loss: 0.2484 - acc: 0.5331
Epoch 7/100
983/983 [==============================] - 0s 230us/step - loss: 0.2481 - acc: 0.5320
Epoch 8/100
983/983 [==============================] - 0s 220us/step - loss: 0.2476 - acc: 0.5310
Epoch 9/100
983/983 [==============================] - 0s 182us/step - loss: 0.2468 - acc: 0.5473
Epoch 10/100
983/983 [==============================] - 0s 150us/step - loss: 0.2468 - acc: 0.5361
Epoch 11/100
983/98

983/983 [==============================] - 0s 195us/step - loss: 0.2020 - acc: 0.6887
Epoch 84/100
983/983 [==============================] - 0s 155us/step - loss: 0.2050 - acc: 0.6734
Epoch 85/100
983/983 [==============================] - 0s 113us/step - loss: 0.2023 - acc: 0.6897
Epoch 86/100
983/983 [==============================] - 0s 125us/step - loss: 0.1998 - acc: 0.6989
Epoch 87/100
983/983 [==============================] - 0s 131us/step - loss: 0.1978 - acc: 0.6918
Epoch 88/100
983/983 [==============================] - 0s 128us/step - loss: 0.2010 - acc: 0.7019
Epoch 89/100
983/983 [==============================] - 0s 112us/step - loss: 0.1989 - acc: 0.6938
Epoch 90/100
983/983 [==============================] - 0s 114us/step - loss: 0.1980 - acc: 0.6968
Epoch 91/100
983/983 [==============================] - 0s 131us/step - loss: 0.1979 - acc: 0.6968
Epoch 92/100
983/983 [==============================] - 0s 126us/step - loss: 0.1985 - acc: 0.6958
Epoch 93/100
983/983 [=

In [11]:
### https://www.quora.com/What-is-the-difference-between-keras-evaluate-and-keras-predict
classifier.evaluate(X_test, y_test)

246/246 [==============================] - 0s 529us/step


[0.34984203010070614, 0.4430894304097183]

In [ ]:
### SVM

#get stock
df_stock = extract_stock(stock_name)
    
#prepare variables
df_stock['H-L'] = df_stock['high'] - df_stock['low']
df_stock['O-C'] = df_stock['close'] - df_stock['open']
df_stock['3day MA'] = df_stock['close'].shift(1).rolling(window = 3).mean()
df_stock['10day MA'] = df_stock['close'].shift(1).rolling(window = 10).mean()
df_stock['30day MA'] = df_stock['close'].shift(1).rolling(window = 30).mean()
df_stock['Std_dev']= df_stock['close'].rolling(5).std()
    
#prepare predicted
df_stock['Price_Rise'] = np.where(df_stock['close'].shift(-1) > df_stock['close'], 1, 0)
    
#remove na after shift
df_stock = df_stock.dropna()
    
#define X
X = df_stock.iloc[:, 4:-1]
    
#define y
y = np.where(df_stock['close'].shift(-1) > df_stock['close'],1,-1)
    
#define split
split_percentage = 0.8
split = int(len(df_stock)*split_percentage)
    
#prepare train and test sets
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]
    
#define and teach model
cls = SVC().fit(X_train, y_train)
    
#create metrics
accuracy_train_svm = accuracy_score(y_train, cls.predict(X_train))
accuracy_test_svm = accuracy_score(y_test, cls.predict(X_test))
    
#calculate returns
df_stock['Predicted_Signal'] = cls.predict(X)
df_stock['Market_Returns'] = np.log(df_stock['close']/df_stock['close'].shift(1))
Cumulative_Market_Returns = df_stock[split:]['Market_Returns'].cumsum()*100

df_stock['Strategy_Returns'] = df_stock['Market_Returns'] * df_stock['Predicted_Signal'].shift(1)
Cumulative_Strategy_Returns = df_stock[split:]['Strategy_Returns'].cumsum()*100

Cumulative_SVM_Returns = Cumulative_Strategy_Returns

#calculate sharpe ratio
Std = Cumulative_Strategy_Returns.std()
Sharpe = (Cumulative_Strategy_Returns-Cumulative_Market_Returns)/Std
Sharpe_SVM = Sharpe.mean()
    
#return Sharpe_SVM

In [ ]:
# KNN

#get stock
df_stock = extract_stock(stock_name)
    
#prepare variables
df_stock['H-L'] = df_stock['high'] - df_stock['low']
df_stock['O-C'] = df_stock['close'] - df_stock['open']
df_stock['3day MA'] = df_stock['close'].shift(1).rolling(window = 3).mean()
df_stock['10day MA'] = df_stock['close'].shift(1).rolling(window = 10).mean()
df_stock['30day MA'] = df_stock['close'].shift(1).rolling(window = 30).mean()
df_stock['Std_dev']= df_stock['close'].rolling(5).std()
    
#prepare predicted
df_stock['Price_Rise'] = np.where(df_stock['close'].shift(-1) > df_stock['close'],1,-1)
    
#remove na after shift
df_stock = df_stock.dropna()
    
#define X
X = df_stock.iloc[:, 4:-1]
    
#define y
y = np.where(df_stock['close'].shift(-1) > df_stock['close'],1,-1)
    
#define split
split_percentage = 0.8
split = int(split_percentage*len(df_stock))

#prepare train and test sets
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]
    
#define and teach model
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train, y_train)
    
#create metrics
accuracy_train_knn = accuracy_score(y_train, knn.predict(X_train))
accuracy_test_knn = accuracy_score(y_test, knn.predict(X_test))
    
#calculate returns
df_stock['Predicted_Signal'] = knn.predict(X)
df_stock['Market_Returns'] = np.log(df_stock['close']/df_stock['close'].shift(1))
Cumulative_Market_Returns = df_stock[split:]['Market_Returns'].cumsum()*100

df_stock['Strategy_Returns'] = df_stock['Market_Returns'] * df_stock['Predicted_Signal'].shift(1)
Cumulative_Strategy_Returns = df_stock[split:]['Strategy_Returns'].cumsum()*100

Cumulative_KNN_Returns = Cumulative_Strategy_Returns

#calculate sharpe ratio
Std = Cumulative_Strategy_Returns.std()
Sharpe = (Cumulative_Strategy_Returns-Cumulative_Market_Returns)/Std
Sharpe_KNN = Sharpe.mean()
    
#return Sharpe_KNN

In [ ]:
### LR

#get stock
df_stock = extract_stock(stock_name)
    
#prepare variables
df_stock['H-L'] = df_stock['high'] - df_stock['low']
df_stock['O-C'] = df_stock['close'] - df_stock['open']
df_stock['3day MA'] = df_stock['close'].shift(1).rolling(window = 3).mean()
df_stock['10day MA'] = df_stock['close'].shift(1).rolling(window = 10).mean()
df_stock['30day MA'] = df_stock['close'].shift(1).rolling(window = 30).mean()
df_stock['Std_dev']= df_stock['close'].rolling(5).std()
df_stock = df_stock.dropna()
    
#prepare predicted
df_stock['Price_Rise'] = np.where(df_stock['close'].shift(-1) > df_stock['close'], 1, 0)
    
#define X
X = df_stock.iloc[:, 4:-1]
    
#define y
y = np.where(df_stock['close'].shift(-1) > df_stock['close'],1,-1)

#define split
split_percentage = 0.8
split = int(split_percentage*len(df_stock))
    
#prepare train and test sets
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]
    
#define and teach model
model = LogisticRegression()
model = model.fit(X_train,y_train)
    
#create metrics
accuracy_train_lr = accuracy_score(y_train, model.predict(X_train))
accuracy_test_lr = accuracy_score(y_test, model.predict(X_test))

pd.DataFrame(list(zip(X.columns, np.transpose(model.coef_))))
probability = model.predict_proba(X_test)
predicted = model.predict(X_test)
probability = model.predict_proba(X_test)
metrics.confusion_matrix(y_test, predicted)
metrics.classification_report(y_test, predicted)
model.score(X_test,y_test)
cross_val = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
cross_val.mean()
    
#calculate returns
df_stock['Predicted_Signal'] = model.predict(X)
df_stock['Market_Returns'] = np.log(df_stock['close']/df_stock['close'].shift(1))
Cumulative_Market_Returns = df_stock[split:]['Market_Returns'].cumsum()*100

df_stock['Strategy_Returns'] = df_stock['Market_Returns'] * df_stock['Predicted_Signal'].shift(1)
Cumulative_Strategy_Returns = df_stock[split:]['Strategy_Returns'].cumsum()*100

Cumulative_LR_Returns = Cumulative_Strategy_Returns

#calculate sharpe ratio
Std = Cumulative_Strategy_Returns.std()
Sharpe = (Cumulative_Strategy_Returns-Cumulative_Market_Returns)/Std
Sharpe_LR = Sharpe.mean()
    
#return Sharpe_LR

In [ ]:
df_stock = df.loc[df['Name'] == stock_name]

df_stock['date'] = pd.to_datetime(df_stock['date'])
df_stock['close'] = pd.to_numeric(df_stock['close'], downcast='float')

df_stock.set_index('date',inplace=True)  
df_stock.sort_index(inplace=True)  

df_stock.head()

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(Cumulative_Market_Returns, color='r', label='Market Returns')
plt.plot(Cumulative_ANN_Returns, color='g', label='ANN Returns')
plt.plot(Cumulative_SVM_Returns, color='b', label='SVM Returns')
plt.plot(Cumulative_KNN_Returns, color='m', label='KNN Returns')
plt.plot(Cumulative_LR_Returns, color='c', label='LR Returns')
plt.grid()
plt.legend()
plt.show()

In [ ]:
df_stock = df_stock.loc['2017-02-01':'2018-02-01']
plt.figure(figsize=(15,8))
plt.plot(df_stock['close'], color='g', label='Close')
plt.grid()
plt.legend()
plt.show()

In [ ]:
print('accuracy_train_knn: ' + str(accuracy_train_knn))
print('accuracy_train_lr: ' + str(accuracy_train_lr))
print('accuracy_train_svm: ' + str(accuracy_train_svm))

print('accuracy_test_knn: ' + str(accuracy_test_knn))
print('accuracy_test_lr: ' + str(accuracy_test_lr))
print('accuracy_test_svm: ' + str(accuracy_test_svm))

In [ ]:
##########################################################################################
################################### 0. ALL SCRIPTS END ###################################
##########################################################################################

In [ ]:
###################################################################################################
################################### 1. START OF NEURAL NETWORKS ###################################
###################################################################################################

In [ ]:
def ANN(df, stock_name):
    
    #get stock
    df_stock = extract_stock(stock_name)
    
    #prepare variables
    df_stock['H-L'] = df_stock['high'] - df_stock['low']
    df_stock['O-C'] = df_stock['close'] - df_stock['open']
    df_stock['3day MA'] = df_stock['close'].shift(1).rolling(window = 3).mean()
    df_stock['10day MA'] = df_stock['close'].shift(1).rolling(window = 10).mean()
    df_stock['30day MA'] = df_stock['close'].shift(1).rolling(window = 30).mean()
    df_stock['Std_dev']= df_stock['close'].rolling(5).std()
    
    #prepare predicted
    df_stock['Price_Rise'] = np.where(df_stock['close'].shift(-1) > df_stock['close'], 1, 0)
    
    #remove na after shift
    df_stock = df_stock.dropna()
    
    #define X
    X = df_stock.iloc[:, 4:-1]
    
    #define y
    y = df_stock.iloc[:, -1]
    
    #define split
    split_percentage = 0.8
    split = int(len(df_stock)*split_percentage)
    
    #prepare train and test sets
    X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]
    
    #define scaler
    sc = StandardScaler()
    
    #scale-replace X_train and X_test
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    #define model
    classifier = Sequential()
    classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', input_dim = X.shape[1]))
    classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
    
    #fit model
    classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, verbose = 0)
    
    y_pred = classifier.predict(X_test)
    y_pred = (y_pred > 0.5)
    
    df_stock['y_pred'] = np.NaN
    df_stock.iloc[(len(df_stock) - len(y_pred)):,-1:] = y_pred
    trade_dataset = df_stock.dropna()
    
    #create metrics
    
    #calculate returns
    df_stock['Tomorrows Returns'] = 0.
    df_stock['Tomorrows Returns'] = np.log(df_stock['close']/df_stock['close'].shift(1))
    df_stock['Tomorrows Returns'] = df_stock['Tomorrows Returns'].shift(-1)

    df_stock['Strategy Returns'] = 0.
    df_stock['Strategy Returns'] = np.where(df_stock['y_pred'] == True, df_stock['Tomorrows Returns'], - df_stock['Tomorrows Returns'])

    df_stock['Cumulative Market Returns'] = np.cumsum(df_stock['Tomorrows Returns'])
    df_stock['Cumulative Strategy Returns'] = np.cumsum(df_stock['Strategy Returns'])

    Cum_Strategy_Returns = df_stock['Cumulative Strategy Returns']
    Cumulative_Market_Returns = df_stock['Cumulative Market Returns']
    
    #calculate sharpe ratio
    Std = Cum_Strategy_Returns.std()
    Sharpe = (Cum_Strategy_Returns-Cumulative_Market_Returns)/Std
    Sharpe_ANN = Sharpe.mean()
    
    return Sharpe_ANN

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(Cumulative_Strategy_Returns, color='g', label='Strategy Returns')
plt.plot(Cumulative_Market_Returns, color='r', label='Market Returns')
plt.legend()
plt.show()

In [ ]:
#################################################################################################
################################### 1. END OF NEURAL NETWORKS ###################################
#################################################################################################

In [ ]:
###################################################################################################
################################### 2. STARY OF SUPPORT VECTORS ###################################
###################################################################################################

In [ ]:
def SVM(df, stock_name):
    
    #get stock
    df_stock = extract_stock(stock_name)
    
    #prepare variables
    df_stock['H-L'] = df_stock['high'] - df_stock['low']
    df_stock['O-C'] = df_stock['close'] - df_stock['open']
    df_stock['3day MA'] = df_stock['close'].shift(1).rolling(window = 3).mean()
    df_stock['10day MA'] = df_stock['close'].shift(1).rolling(window = 10).mean()
    df_stock['30day MA'] = df_stock['close'].shift(1).rolling(window = 30).mean()
    df_stock['Std_dev']= df_stock['close'].rolling(5).std()
    
    #prepare predicted
    df_stock['Price_Rise'] = np.where(df_stock['close'].shift(-1) > df_stock['close'], 1, 0)
    
    #remove na after shift
    df_stock = df_stock.dropna()
    
    #define X
    X = df_stock.iloc[:, 4:-1]
    
    #define y
    y = np.where(df_stock['close'].shift(-1) > df_stock['close'],1,-1)
    
    #define split
    split_percentage = 0.8
    split = int(len(df_stock)*split_percentage)
    
    #prepare train and test sets
    X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]
    
    #define and teach model
    cls = SVC().fit(X_train, y_train)
    
    #create metrics
    accuracy_train_SVM = accuracy_score(y_train, cls.predict(X_train))
    accuracy_test_SVM = accuracy_score(y_test, cls.predict(X_test))
    
    #calculate returns
    df_stock['Predicted_Signal'] = cls.predict(X)
    df_stock['Market_Returns'] = np.log(df_stock['close']/df_stock['close'].shift(1))
    Cumulative_Market_Returns = df_stock[split:]['Market_Returns'].cumsum()*100

    df_stock['Strategy_Returns'] = df_stock['Market_Returns'] * df_stock['Predicted_Signal'].shift(1)
    Cumulative_Strategy_Returns = df_stock[split:]['Strategy_Returns'].cumsum()*100
    
    #calculate sharpe ratio
    Std = Cumulative_Strategy_Returns.std()
    Sharpe = (Cumulative_Strategy_Returns-Cumulative_Market_Returns)/Std
    Sharpe_SVM = Sharpe.mean()
    
    return (Sharpe_SVM,accuracy_train_SVM,accuracy_test_SVM)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(Cumulative_Strategy_Returns, color='g', label='Strategy Returns')
plt.plot(Cumulative_Market_Returns, color='r', label='Market Returns')
plt.legend()
plt.show()

In [ ]:
############################################################
################################### 2. END OF SUPPORT VECTORS ###################################
#################################################################################################

In [ ]:
#######################################################################################
################################### 3. START OF KNN ###################################
#######################################################################################

In [ ]:
def KNN(df, stock_name):
    
    #get stock
    df_stock = extract_stock(stock_name)
    
    #prepare variables
    df_stock['H-L'] = df_stock['high'] - df_stock['low']
    df_stock['O-C'] = df_stock['close'] - df_stock['open']
    df_stock['3day MA'] = df_stock['close'].shift(1).rolling(window = 3).mean()
    df_stock['10day MA'] = df_stock['close'].shift(1).rolling(window = 10).mean()
    df_stock['30day MA'] = df_stock['close'].shift(1).rolling(window = 30).mean()
    df_stock['Std_dev']= df_stock['close'].rolling(5).std()
    
    #prepare predicted
    df_stock['Price_Rise'] = np.where(df_stock['close'].shift(-1) > df_stock['close'],1,-1)
    
    #remove na after shift
    df_stock = df_stock.dropna()
    
    #define X
    X = df_stock.iloc[:, 4:-1]
    
    #define y
    y = np.where(df_stock['close'].shift(-1) > df_stock['close'],1,-1)
    
    #define split
    split_percentage = 0.8
    split = int(split_percentage*len(df_stock))

    #prepare train and test sets
    X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]
    
    #define and teach model
    knn = KNeighborsClassifier(n_neighbors=15)
    knn.fit(X_train, y_train)
    
    #create metrics
    accuracy_train_KNN = accuracy_score(y_train, knn.predict(X_train))
    accuracy_test_KNN = accuracy_score(y_test, knn.predict(X_test))
    
    #calculate returns
    df_stock['Predicted_Signal'] = knn.predict(X)
    df_stock['Market_Returns'] = np.log(df_stock['close']/df_stock['close'].shift(1))
    Cumulative_Market_Returns = df_stock[split:]['Market_Returns'].cumsum()*100

    df_stock['Strategy_Returns'] = df_stock['Market_Returns'] * df_stock['Predicted_Signal'].shift(1)
    Cumulative_Strategy_Returns = df_stock[split:]['Strategy_Returns'].cumsum()*100
    
    #calculate sharpe ratio
    Std = Cumulative_Strategy_Returns.std()
    Sharpe = (Cumulative_Strategy_Returns-Cumulative_Market_Returns)/Std
    Sharpe_KNN = Sharpe.mean()
    
    return (Sharpe_KNN,accuracy_train_KNN,accuracy_test_KNN)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(Cumulative_Market_Returns, color='r', label='Market Returns')
plt.plot(Cumulative_Strategy_Returns, color='g', label='Strategy Retruns')
plt.legend()
plt.show()

In [ ]:
#####################################################################################
################################### 3. END OF KNN ###################################
#####################################################################################

In [ ]:
#######################################################################################################
################################### 4. START OF LOGISTIC REGRESSION ###################################
#######################################################################################################

In [ ]:
def LR(df, stock_name):
    
    #get stock
    df_stock = extract_stock(stock_name)
    
    #prepare variables
    df_stock['H-L'] = df_stock['high'] - df_stock['low']
    df_stock['O-C'] = df_stock['close'] - df_stock['open']
    df_stock['3day MA'] = df_stock['close'].shift(1).rolling(window = 3).mean()
    df_stock['10day MA'] = df_stock['close'].shift(1).rolling(window = 10).mean()
    df_stock['30day MA'] = df_stock['close'].shift(1).rolling(window = 30).mean()
    df_stock['Std_dev']= df_stock['close'].rolling(5).std()
    df_stock = df_stock.dropna()
    
    #prepare predicted
    df_stock['Price_Rise'] = np.where(df_stock['close'].shift(-1) > df_stock['close'], 1, 0)
    
    #define X
    X = df_stock.iloc[:, 4:-1]
    
    #define y
    y = np.where(df_stock['close'].shift(-1) > df_stock['close'],1,-1)

    #define split
    split_percentage = 0.8
    split = int(split_percentage*len(df_stock))
    
    #prepare train and test sets
    X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]
    
    #define and teach model
    model = LogisticRegression()
    model = model.fit(X_train,y_train)
    
    #create metrics
    accuracy_train_LR = accuracy_score(y_train, model.predict(X_train))
    accuracy_test_LR = accuracy_score(y_test, model.predict(X_test))
    
    pd.DataFrame(list(zip(X.columns, np.transpose(model.coef_))))
    probability = model.predict_proba(X_test)
    predicted = model.predict(X_test)
    probability = model.predict_proba(X_test)
    metrics.confusion_matrix(y_test, predicted)
    metrics.classification_report(y_test, predicted)
    model.score(X_test,y_test)
    cross_val = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
    cross_val.mean()
    
    #calculate returns
    df_stock['Predicted_Signal'] = model.predict(X)
    df_stock['Market_Returns'] = np.log(df_stock['close']/df_stock['close'].shift(1))
    Cumulative_Market_Returns = df_stock[split:]['Market_Returns'].cumsum()*100

    df_stock['Strategy_Returns'] = df_stock['Market_Returns'] * df_stock['Predicted_Signal'].shift(1)
    Cumulative_Strategy_Returns = df_stock[split:]['Strategy_Returns'].cumsum()*100
    
    #calculate sharpe ratio
    Std = Cumulative_Strategy_Returns.std()
    Sharpe = (Cumulative_Strategy_Returns-Cumulative_Market_Returns)/Std
    Sharpe_LR = Sharpe.mean()
    
    return (Sharpe_LR,accuracy_train_LR,accuracy_test_LR)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(Cumulative_Market_Returns, color='r',label = 'Market Returns')
plt.plot(Cumulative_Strategy_Returns, color='g', label = 'Strategy Returns')
plt.legend()
plt.show()

In [ ]:
#####################################################################################################
################################### 4. END OF LOGISTIC REGRESSION ###################################
#####################################################################################################

In [ ]:
print('Sharpe ANN ratio: %.2f' %Sharpe_ANN)
print('Sharpe SVM ratio: %.2f' %Sharpe_SVM.)
print('Sharpe KNN ratio: %.2f' %Sharpe_KNN)
print('Sharpe LR ratio: %.2f' %Sharpe_LR)

In [ ]:
i=1
for stock_name in stock_names:
    
    print('============= ' + str(i) + ' ===============')
    print('Analysis of ' + stock_name + ' started.')
    
    print('- ANN START -')
    sharpe_ann = ANN(df, stock_name)
    print('Sharpe ANN ratio: %.2f' %sharpe_ann)
    print('- ANN FINISH -')
        
    print('- SVM START -')
    sharpe_svm = SVM(df, stock_name)
    print('Sharpe SVM ratio: %.2f' %sharpe_svm[0])
    print('- SVM FINISH -')
      
    print('- KNN START -')
    sharpe_knn = KNN(df, stock_name)
    print('Sharpe KNN ratio: %.2f' %sharpe_knn[0])
    print('- KNN FINISH -')
       
    print('- LR START -')
    sharpe_lr = LR(df, stock_name)
    print('Sharpe LR ratio: %.2f' %sharpe_lr[0])
    print('- LR FINISH -')
        
    print('Analysis of ' + stock_name + ' finished.')   
    print('Dumping results to dataframe')

    sharpes = pd.DataFrame(np.array([[stock_name, sharpe_ann, sharpe_svm[0], sharpe_knn[0], sharpe_lr[0],sharpe_svm[1], sharpe_svm[2], sharpe_knn[1], sharpe_knn[2], sharpe_lr[1], sharpe_lr[2]]]),
                            columns=['STOCK','ANN','SVM','KNN','LR','acc_SVM_train','acc_SVM_test','acc_KNN_train','acc_KNN_test','acc_LR_train','acc_LR_test']).append(sharpes, ignore_index=True)
    i = i+1

In [ ]:
col_names = list(sharpes_copy)

In [ ]:
for name in col_names:
    if name != 'STOCK':
        sharpes_copy[name] = sharpes_copy[name].astype(float)

In [ ]:
sharpes_copy.round(3)